In [1]:
import warnings
warnings.filterwarnings('ignore')
import sys

import keras
import numpy as np
from keras_preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, MaxPooling2D, BatchNormalization
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils import class_weight

sys.path.append( '../../src')
import counting_functions as mcf
import marsican_functions as msn

Using TensorFlow backend.


In [2]:
datagen = ImageDataGenerator(rotation_range=30, 
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             horizontal_flip=True,
                             vertical_flip=True)

if 'win32' in sys.platform:
    data_directory='F:\\Segmentation_Data\\'
    full_training_directory = data_directory + 'Labelled_imgs\\stratified_data\\training\\'
    full_validation_directory = data_directory +'Labelled_imgs\\stratified_data\\validation\\'

if 'darwin' in sys.platform:
    data_directory='/Volumes/Samsung_T5/Segmentation_Data/'
    full_training_directory = data_directory + 'Labelled_imgs/stratified_data/training/'
    full_validation_directory = data_directory +'Labelled_imgs/stratified_data/validation/'


## Start Modeling

In [3]:
batch_size = 32
train_generator = datagen.flow_from_directory(full_training_directory, 
                                              target_size=(128, 128), 
                                              color_mode="rgb", 
                                              batch_size=batch_size)
validation_generator = datagen.flow_from_directory(full_validation_directory,
                                                   target_size=(128, 128), 
                                                   color_mode='rgb',
                                                   batch_size=batch_size, 
                                                   shuffle=False)
num_classes = len(train_generator.class_indices)
num_test_samples = 8984

class_weights = class_weight.compute_class_weight(
           'balanced',
            np.unique(train_generator.classes), 
            train_generator.classes)

Found 21555 images belonging to 8 classes.
Found 9236 images belonging to 8 classes.


In [4]:
model = Sequential()
model.add(Conv2D(20, (5,5), activation='relu', input_shape=(128,128,3)))
model.add(BatchNormalization(momentum=0.9))
model.add(MaxPooling2D())
model.add(Conv2D(50, (5,5), activation='relu'))
model.add(BatchNormalization(momentum=0.9))
model.add(MaxPooling2D())
model.add(Conv2D(100, (4,4), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(200, (4,4), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(500))
model.add(Dropout(rate=0.25))
model.add(Dense(num_classes))
model.add(Activation('softmax'))


W0830 23:00:44.752855  8920 deprecation_wrapper.py:119] From C:\Users\acsch\Anaconda3\envs\Tensorflow-GPU\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0830 23:00:44.778785  8920 deprecation_wrapper.py:119] From C:\Users\acsch\Anaconda3\envs\Tensorflow-GPU\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0830 23:00:44.786794  8920 deprecation_wrapper.py:119] From C:\Users\acsch\Anaconda3\envs\Tensorflow-GPU\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0830 23:00:44.810701  8920 deprecation_wrapper.py:119] From C:\Users\acsch\Anaconda3\envs\Tensorflow-GPU\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_de

In [5]:
sgd = keras.optimizers.SGD(lr=0.01,
                          momentum=0.9,
                          decay=0.0001)

# Testing Model

In [24]:

model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator,
                    verbose=1, 
                    steps_per_epoch=1000, 
                    epochs=1, 
                    validation_data=validation_generator,
                   validation_steps=300)

Epoch 1/1
1000/1000 [==============================] - 122s 122ms/step - loss: 1.0679 - acc: 0.6055 - val_loss: 0.7752 - val_acc: 0.7053


In [25]:
Y_pred = model.predict_generator(validation_generator, num_test_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['0_colonies','1_colony', 
                '2_colonies', '3_colonies',
                '4_colonies', '5_colonies', 
                '6_colonies', 'outlier']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))


Confusion Matrix
[[1762 1148  977   18  222   68   46]
 [ 766   52  162    8  201  121  286]
 [1101   36    1    0    0    0    1]
 [ 554   10    3    0    0    0    1]
 [ 282    4    0    0    0    0    2]
 [ 300    4    0    0    0    0    0]
 [ 382    3    2    0    0    0    1]]
Classification Report
              precision    recall  f1-score   support

    1_colony       0.34      0.42      0.38      4241
  2_colonies       0.04      0.03      0.04      1596
  3_colonies       0.00      0.00      0.00      1139
  4_colonies       0.00      0.00      0.00       568
  5_colonies       0.00      0.00      0.00       288
  6_colonies       0.00      0.00      0.00       304
     outlier       0.00      0.00      0.00       388

    accuracy                           0.21      8524
   macro avg       0.06      0.06      0.06      8524
weighted avg       0.18      0.21      0.19      8524



In [15]:
# https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045

Y_pred = model.predict_generator(validation_generator, num_test_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['1_colony', '2_colonies', '3_colonies', '4_colonies', '5_colonies', '6_colonies', 'outlier']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))


Confusion Matrix
[[4148   77    4    1    0    1   10]
 [ 126 1328  128    6    0    3    5]
 [  34  174  805  116    5    5    0]
 [  15   12  125  331   48   37    0]
 [   5    2   11   98   67  105    0]
 [   7    1    5   22   23  212   34]
 [  40    7    1    0    0    0  340]]
Classification Report
              precision    recall  f1-score   support

    1_colony       0.95      0.98      0.96      4241
  2_colonies       0.83      0.83      0.83      1596
  3_colonies       0.75      0.71      0.73      1139
  4_colonies       0.58      0.58      0.58       568
  5_colonies       0.47      0.23      0.31       288
  6_colonies       0.58      0.70      0.64       304
     outlier       0.87      0.88      0.88       388

    accuracy                           0.85      8524
   macro avg       0.72      0.70      0.70      8524
weighted avg       0.84      0.85      0.84      8524



## Balance Classes

In [6]:
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
balanced_history = model.fit_generator(train_generator,
                                       verbose=1, 
                                       steps_per_epoch=1000, 
                                       epochs=100 , 
                                       validation_data=validation_generator,
                                      validation_steps=300,
                                      class_weight=class_weights)
model.save('balanced_classes.hdf5')

W0822 18:06:15.504884 13624 deprecation_wrapper.py:119] From C:\Users\acsch\Anaconda3\envs\Tensorflow-GPU\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0822 18:06:15.572340 13624 deprecation.py:323] From C:\Users\acsch\Anaconda3\envs\Tensorflow-GPU\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/150
1000/1000 [==============================] - 170s 170ms/step - loss: 1.2578 - acc: 0.5642 - val_loss: 0.9296 - val_acc: 0.6526
Epoch 2/150
1000/1000 [==============================] - 119s 119ms/step - loss: 0.9432 - acc: 0.6464 - val_loss: 0.7786 - val_acc: 0.7135
Epoch 3/150
1000/1000 [==============================] - 119s 119ms/step - loss: 0.8306 - acc: 0.6833 - val_loss: 0.7435 - val_acc: 0.7183
Epoch 4/150
1000/1000 [==============================] - 120s 120ms/step - loss: 0.7664 - acc: 0.7074 - val_loss: 0.6653 - val_acc: 0.7457
Epoch 5/150
1000/1000 [==============================] - 120s 120ms/step - loss: 0.7080 - acc: 0.7277 - val_loss: 0.7551 - val_acc: 0.7076
Epoch 6/150
1000/1000 [==============================] - 120s 120ms/step - loss: 0.6670 - acc: 0.7421 - val_loss: 0.6814 - val_acc: 0.7409
Epoch 7/150
1000/1000 [==============================] - 120s 120ms/step - loss: 0.6365 - acc: 0.7516 - val_loss: 0.6727 - val_acc: 0.7340
Epoch 8/150
1000/1000 [====

1000/1000 [==============================] - 119s 119ms/step - loss: 0.3438 - acc: 0.8679 - val_loss: 0.3621 - val_acc: 0.8697
Epoch 118/150
1000/1000 [==============================] - 119s 119ms/step - loss: 0.3382 - acc: 0.8705 - val_loss: 0.3776 - val_acc: 0.8612
Epoch 119/150
1000/1000 [==============================] - 120s 120ms/step - loss: 0.3386 - acc: 0.8691 - val_loss: 0.4373 - val_acc: 0.8474
Epoch 120/150
1000/1000 [==============================] - 120s 120ms/step - loss: 0.3360 - acc: 0.8694 - val_loss: 0.4420 - val_acc: 0.8357
Epoch 121/150
1000/1000 [==============================] - 120s 120ms/step - loss: 0.3374 - acc: 0.8713 - val_loss: 0.4796 - val_acc: 0.8228
Epoch 122/150
1000/1000 [==============================] - 119s 119ms/step - loss: 0.3315 - acc: 0.8721 - val_loss: 0.3720 - val_acc: 0.8698
Epoch 123/150
1000/1000 [==============================] - 119s 119ms/step - loss: 0.3390 - acc: 0.8709 - val_loss: 0.3509 - val_acc: 0.8738
Epoch 124/150
1000/1000 [==

In [7]:
model = keras.models.load_model('balanced_classes.hdf5')
Y_pred = model.predict_generator(validation_generator, num_test_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['1_colony', '2_colonies', '3_colonies', '4_colonies', '5_colonies', '6_colonies', 'outlier']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))


Confusion Matrix
[[4187   46    2    0    0    0    6]
 [ 141 1348   99    5    0    2    1]
 [  40  194  840   59    4    1    1]
 [  14   15  165  322   36   16    0]
 [   5    2   24  114   82   61    0]
 [   9    1    4   32   47  210    1]
 [  25    9    1    0    0    0  353]]
Classification Report
              precision    recall  f1-score   support

    1_colony       0.95      0.99      0.97      4241
  2_colonies       0.83      0.84      0.84      1596
  3_colonies       0.74      0.74      0.74      1139
  4_colonies       0.61      0.57      0.59       568
  5_colonies       0.49      0.28      0.36       288
  6_colonies       0.72      0.69      0.71       304
     outlier       0.98      0.91      0.94       388

    accuracy                           0.86      8524
   macro avg       0.76      0.72      0.73      8524
weighted avg       0.85      0.86      0.86      8524



In [17]:
model = keras.models.load_model('balanced_classes.hdf5')
Y_pred = model.predict_generator(validation_generator, num_test_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['1_colony', '2_colonies', '3_colonies', '4_colonies', '5_colonies', '6_colonies', 'outlier']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))


Confusion Matrix
[[4243   36    3    0    0    0    3]
 [ 129 1443   59    1    0    0    0]
 [  23  163  845   55    1    2    0]
 [  12   10  164  316   34   14    0]
 [   7    2   18  116   88   55    0]
 [   5    2    4   25   56  209    0]
 [  23    2    0    0    0    1  352]]
Classification Report
              precision    recall  f1-score   support

    1_colony       0.96      0.99      0.97      4285
  2_colonies       0.87      0.88      0.88      1632
  3_colonies       0.77      0.78      0.77      1089
  4_colonies       0.62      0.57      0.59       550
  5_colonies       0.49      0.31      0.38       286
  6_colonies       0.74      0.69      0.72       301
     outlier       0.99      0.93      0.96       378

    accuracy                           0.88      8521
   macro avg       0.78      0.74      0.75      8521
weighted avg       0.87      0.88      0.88      8521



## Imbalanced Classes

In [22]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator,
                    verbose=1, 
                    steps_per_epoch=1000, 
                    epochs=1, 
                    validation_data=validation_generator,
                   validation_steps=300,
                   class_weight=class_weights)

Epoch 1/1
1000/1000 [==============================] - 120s 120ms/step - loss: 0.9240 - acc: 0.6460 - val_loss: 0.8217 - val_acc: 0.7030


In [23]:
Y_pred = model.predict_generator(validation_generator, num_test_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['1_colony', '2_colonies', '3_colonies', '4_colonies', '5_colonies', '6_colonies', 'outlier']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))


Confusion Matrix
[[2740 1107  328   27    0   14   25]
 [ 221  446  550   94   13  257   15]
 [ 760   41   21    3    2   45  267]
 [ 552   15    0    0    0    0    1]
 [ 277   10    0    0    0    0    1]
 [ 290   13    0    0    0    0    1]
 [ 378   10    0    0    0    0    0]]
Classification Report
              precision    recall  f1-score   support

    1_colony       0.53      0.65      0.58      4241
  2_colonies       0.27      0.28      0.28      1596
  3_colonies       0.02      0.02      0.02      1139
  4_colonies       0.00      0.00      0.00       568
  5_colonies       0.00      0.00      0.00       288
  6_colonies       0.00      0.00      0.00       304
     outlier       0.00      0.00      0.00       388

    accuracy                           0.38      8524
   macro avg       0.12      0.13      0.13      8524
weighted avg       0.32      0.38      0.34      8524



## Grayscale

In [4]:
batch_size = 64
train_generator = datagen.flow_from_directory(full_training_directory, 
                                              target_size=(128, 128), 
                                              color_mode="grayscale", 
                                              batch_size=batch_size)
validation_generator = datagen.flow_from_directory(full_validation_directory,
                                                   target_size=(128, 128), 
                                                   color_mode='grayscale',
                                                   batch_size=batch_size, 
                                                   shuffle=False)
num_classes = len(train_generator.class_indices)
num_test_samples = 10025

class_weights = class_weight.compute_class_weight(
           'balanced',
            np.unique(train_generator.classes), 
            train_generator.classes)

Found 23398 images belonging to 8 classes.
Found 10025 images belonging to 8 classes.


In [5]:
model = Sequential()
model.add(Conv2D(20, (5,5), activation='relu', input_shape=(128,128,1)))
model.add(BatchNormalization(momentum=0.9))
model.add(MaxPooling2D())
model.add(Conv2D(50, (5,5), activation='relu'))
model.add(BatchNormalization(momentum=0.9))
model.add(MaxPooling2D())
model.add(Conv2D(100, (4,4), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(200, (4,4), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(500, activation='relu')) # Added Relu, maybe remove
model.add(Dropout(rate=0.25))
model.add(Dense(num_classes))
model.add(Activation('softmax'))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
sgd = keras.optimizers.SGD(lr=0.01,
                          momentum=0.9,
                          decay=0.0001)

In [7]:
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
balanced_history = model.fit_generator(train_generator,
                                       verbose=1, 
                                       steps_per_epoch=500, 
                                       epochs=100, 
                                       validation_data=validation_generator,
                                      validation_steps=num_test_samples // batch_size+1,
                                      class_weight=class_weights)
model.save('balanced_gray_classes.hdf5')

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
500/500 [==============================] - 111s 222ms/step - loss: 1.2661 - acc: 0.5531 - val_loss: 1.0412 - val_acc: 0.6050
Epoch 2/100
500/500 [==============================] - 55s 109ms/step - loss: 1.0078 - acc: 0.6213 - val_loss: 0.9206 - val_acc: 0.6505
Epoch 3/100
500/500 [==============================] - 55s 109ms/step - loss: 0.8690 - acc: 0.6665 - val_loss: 0.7766 - val_acc: 0.6980
Epoch 4/100
500/500 [==============================] - 54s 109ms/step - loss: 0.7651 - acc: 0.7069 - val_loss: 0.7547 - val_acc: 0.7298
Epoch 5/100
500/500 [==============================] - 55s 109ms/step - loss: 0.6882 - acc: 0.7364 - val_loss: 0.5943 - val_acc: 0.7775
Epoch 6/100
500/500 [==============================] - 54s 109ms/step - loss: 0.6436 - acc: 0.7538 - val_loss: 0.5828 - val_acc: 0.7796
Epoch 7/100
500/500 [==============================] - 54s 109ms/step - loss: 0.6116 - acc: 0.7665 - val_loss: 0.5511 - val_acc: 0.7884

Epoch 59/100
500/500 [==============================] - 54s 109ms/step - loss: 0.3241 - acc: 0.8743 - val_loss: 0.3178 - val_acc: 0.8794
Epoch 60/100
500/500 [==============================] - 54s 109ms/step - loss: 0.3249 - acc: 0.8762 - val_loss: 0.3201 - val_acc: 0.8801
Epoch 61/100
500/500 [==============================] - 54s 109ms/step - loss: 0.3185 - acc: 0.8777 - val_loss: 0.3208 - val_acc: 0.8818
Epoch 62/100
500/500 [==============================] - 54s 109ms/step - loss: 0.3238 - acc: 0.8760 - val_loss: 0.3137 - val_acc: 0.8832
Epoch 63/100
500/500 [==============================] - 55s 110ms/step - loss: 0.3158 - acc: 0.8785 - val_loss: 0.3207 - val_acc: 0.8803
Epoch 64/100
500/500 [==============================] - 55s 109ms/step - loss: 0.3156 - acc: 0.8787 - val_loss: 0.3153 - val_acc: 0.8805
Epoch 65/100
500/500 [==============================] - 54s 109ms/step - loss: 0.3204 - acc: 0.8760 - val_loss: 0.3125 - val_acc: 0.8831
Epoch 66/100
500/500 [===================

In [8]:
model2 = keras.models.load_model('balanced_gray_classes.hdf5')
validation_generator.reset()
Y_pred = model2.predict_generator(validation_generator, num_test_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['0_colonies','1_colony', '2_colonies', '3_colonies', '4_colonies', '5_colonies', '6_colonies', 'outlier']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))


Confusion Matrix
[[ 439    0    0    0    0    0    0    0]
 [   3 4981   75    4    1    0    0   10]
 [   1  150 1698   54    2    0    0    3]
 [   1   36  176  824   47    4    1    0]
 [   0   16    6  170  294   53   11    0]
 [   0    3    2   19   93  122   46    1]
 [   0   11    1    7   19   82  179    2]
 [   1   16    2    1    0    0    0  358]]
Classification Report
              precision    recall  f1-score   support

  0_colonies       0.99      1.00      0.99       439
    1_colony       0.96      0.98      0.97      5074
  2_colonies       0.87      0.89      0.88      1908
  3_colonies       0.76      0.76      0.76      1089
  4_colonies       0.64      0.53      0.58       550
  5_colonies       0.47      0.43      0.45       286
  6_colonies       0.76      0.59      0.67       301
     outlier       0.96      0.95      0.95       378

    accuracy                           0.89     10025
   macro avg       0.80      0.77      0.78     10025
weighted avg       0

In [26]:
balanced_history.history['acc']

[0.8799199674905183]

In [27]:
Y_pred

array([[5.8091256e-15, 9.7918092e-18, 4.0638271e-10, ..., 3.7677091e-01,
        6.7156717e-02, 7.4900477e-18],
       [2.8744179e-12, 4.4079595e-14, 1.3031728e-05, ..., 5.1736284e-02,
        2.4789143e-03, 2.2783513e-14],
       [7.0208116e-05, 1.5387968e-04, 3.9175007e-02, ..., 7.6512583e-02,
        2.2225235e-02, 1.0948159e-05],
       ...,
       [8.7583613e-10, 4.8992523e-16, 2.5101923e-07, ..., 2.5697449e-01,
        6.1039466e-02, 3.1508889e-11],
       [1.7918350e-17, 3.5399584e-23, 4.2578211e-12, ..., 5.0740635e-01,
        8.9521043e-02, 9.5164997e-20],
       [2.8162518e-18, 7.4315928e-25, 2.4202788e-19, ..., 1.2741689e-01,
        8.6996293e-01, 1.4945137e-18]], dtype=float32)

In [9]:
import tensorflow as tf; print(tf.__version__)

1.13.1
